In [ ]:
!cp ../input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
import gdcm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import glob
from PIL import Image
from joblib import Parallel, delayed
from tqdm import tqdm
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import tensorflow as tf
import tensorflow_io as tfio

In [ ]:
fp = '../input/rsna-breast-cancer-detection/train_images/10006/462822612.dcm'

# pydicom read pixel array

In [ ]:
%%time
pydicom_img = pydicom.dcmread(fp).pixel_array
pydicom_img.shape

# tfio read dicom

In [ ]:
%%time
image_bytes = tf.io.read_file(fp)
tfio_image = 65535 - tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.uint16).numpy().squeeze()
tfio_image.shape

# tfio read scaled to 255 (lossy on error) dicom

In [ ]:
%%time
image_bytes = tf.io.read_file(fp)
tfio_lossy_image = 255 - tfio.image.decode_dicom_image(image_bytes, scale='auto', on_error='lossy', dtype=tf.uint8).numpy().squeeze()
tfio_lossy_image.shape

# maximum values

In [ ]:
print (pydicom_img.max())
print (tfio_image.max())
print (tfio_lossy_image.max())

In [ ]:
%%time
fig, axes = plt.subplots(1,3, figsize=(10,10))
axes[0].imshow(pydicom_img, cmap='gray')
axes[0].set_title('image')
axes[1].imshow(tfio_image, cmap='gray')
axes[1].set_title('tfio image');
axes[2].imshow(tfio_lossy_image, cmap='gray')
axes[2].set_title('tfio image scaled');

# Pydicom Read and Crop

In [ ]:
%%time
pydicom_img = pydicom.dcmread(fp).pixel_array
pydicom_img = pydicom_img[:,pydicom_img.std(axis=0) > 60]
pydicom_img = pydicom_img[pydicom_img.std(axis=1) > 60, :]
pydicom_img.shape

# TFIO Read and Crop

In [ ]:
%%time
image_bytes = tf.io.read_file(fp)
tfio_image = 65535 - tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.uint16).numpy().squeeze()
tfio_image = tfio_image[:,tfio_image.std(axis=0) > 60]
tfio_image = tfio_image[tfio_image.std(axis=1) > 60, :]
tfio_image.shape

# TFIO Scaled Read and Crop

In [ ]:
%%time
image_bytes = tf.io.read_file(fp)
tfio_lossy_image = 255 - tfio.image.decode_dicom_image(image_bytes, scale='auto', on_error='lossy', dtype=tf.uint8).numpy().squeeze()
tfio_lossy_image = tfio_lossy_image[:,tfio_lossy_image.std(axis=0) > 0.5]
tfio_lossy_image = tfio_lossy_image[tfio_lossy_image.std(axis=1) > 0.5, :]
tfio_lossy_image.shape

In [ ]:
%%time
fig, axes = plt.subplots(1,3, figsize=(10,10))
axes[0].imshow(pydicom_img, cmap='gray')
axes[0].set_title('image')
axes[1].imshow(tfio_image, cmap='gray')
axes[1].set_title('tfio image');
axes[2].imshow(tfio_lossy_image, cmap='gray')
axes[2].set_title('tfio lossy image');

In [ ]:
nan